In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
PATH_TO_DATA = 'hw3_data'

In [3]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [4]:
data = pd.concat([pd.read_json(file, lines=True, encoding='utf-8') for file in files], axis=0, ignore_index=True)

In [5]:
data.shape

(1987, 5)

In [6]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

In [7]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


## Токенизация, удаление стоп-слов и нормализация.

In [8]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [9]:
data['content_norm'] = data['content'].apply(normalize)

In [10]:
data['title_norm'] = data['title'].apply(normalize)

In [11]:
data['title_norm'].head(10)

0    [молодёжный, яблоко, оппозиционный, деятельнос...
1                                   [газпром, хватить]
2        [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, о...
4    [новый, москва, остаться, территория, экологич...
5    [f1, гран-при, сша, пройти, четыре, машина, ст...
6    [100, ведущий, политик, россия, февраль, 2018,...
7             [закон, культура, принимать, фон, арест]
8    [насколько, реальный, газовый, подоплёка, сири...
9    [фсб, калужский, область, задержать, четверо, ...
Name: title_norm, dtype: object

In [17]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [12]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


In [13]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [яблоко, молодёжный, который, акция, год, акти...
1     [миллиард, газа, год, куб, метр, газпром, добы...
2     [год, это, книга, роман, тот, писать, выйти, о...
3     [ким, зинаида, видео, год, журналист, суд, дел...
4     [площадь, территория, новый, москва, га, котор...
5     [гонка, который, команда, место, позиция, два,...
6     [место, влияние, рф, позиция, глава, россия, п...
7     [культура, закон, который, сфера, стд, разрабо...
8     [газопровод, сирия, год, турция, газа, россия,...
9     [участник, рф, террористический, организация, ...
10    [тихон, стихомант, это, журибеда, старик, врем...
11    [жильё, метр, кв, год, миллион, жилищный, насе...
12    [российский, театр, зритель, стать, кино, год,...
13    [фестиваль, революция, фильм, ленин, награда, ...
14    [вечер, фёдоров, балашов, книга, который, викт...
15    [который, сирийский, переговоры, резолюция, оо...
16    [рф, суд, кс, запрос, постановление, право, че...
17    [электромобиль, год, тысяча, кобальт, кото

In [14]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [15]:
data['content_norm'] = data['content'].apply(normalize)

In [16]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


In [17]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [яблоко, акция, год, активист, деятельность, п...
1    [миллиард, газа, год, куб, метр, газпром, добы...
2    [год, книга, роман, мир, перевод, стихотворени...
3    [ким, зинаида, видео, год, журналист, суд, дел...
4    [площадь, территория, москва, га, столица, тин...
5    [гонка, команда, место, позиция, круг, чемпион...
6    [место, влияние, рф, позиция, глава, россия, п...
7    [культура, закон, сфера, концепция, проект, из...
8    [газопровод, сирия, год, турция, газа, россия,...
9    [участник, рф, организация, государство, облас...
Name: content_norm, dtype: object

In [18]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [19]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [20]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [21]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [22]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [23]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [24]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'молодая человек',
  'политика',
  'виктор',
  'тимур',
  'репрессия'],
 ['миллиард куб',
  'куб метр',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'добыча газа',
  'метр',
  'холдинг'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'читатель',
  'перевод',
  'год',
  'поэзия',
  'произведение',
  'том']]

In [25]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Baseline:  
Precision -  0.13  
Recall -  0.24  
F1 -  0.16  
Jaccard -  0.09  

## Попытки привлечь сторонние библиотеки: rutermextract, multi_rake

In [32]:
data.head()

,content,keywords,summary,title,url,content_norm,title_norm,content_norm_str
0,"Многие интересуются, зачем нужна «Яблоку» моло...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,"[яблоко, фракция, задача, яблоко, привлечение,...","[молодёжный, яблоко, оппозиционный, деятельнос...",яблоко фракция задача яблоко привлечение молод...
1,Вчера «Газпром» снизил верхнюю планку прогноза...,"[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,"[газпром, планка, прогноз, добыча, газа, год, ...","[газпром, хватить]",газпром планка прогноз добыча газа год год кон...
2,Долголетний труд Евгения Витковского на ниве п...,"[франсуа рабле, сервантес, шекспир, конан дойл...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,"[труд, евгений, витковский, нива, перевод, кач...","[бесконечный, партия, четырехмерный, шахматы]",труд евгений витковский нива перевод качество ...
3,В Ленинском районном суде продолжаются слушани...,"[владивосток, суд, ким, футина, выборы, боевое...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,"[суд, слушание, дело, экс-депутат, дума, влади...","[экс-депутат, осудить, фальсификация, выбор, о...",суд слушание дело экс-депутат дума владивосток...
4,В 2012 году российская столица резко увеличила...,"[новая москва, подмосковье, благоустройство, т...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,"[год, столица, размер, результат, присоединени...","[новый, москва, остаться, территория, экологич...",год столица размер результат присоединение час...


In [33]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

In [74]:
def extract_terms(text):
    l=[]
    for term in term_extractor(text):
        if term.count>=3:
            l.append(term.normalized)
    return l

In [75]:
data['term_extract']=data['content'].apply(extract_terms)

C:\Users\vasel\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [76]:
evaluate(data['keywords'], data['term_extract'])

Precision -  0.16
Recall -  0.21
F1 -  0.15
Jaccard -  0.09


In [26]:
from multi_rake import Rake
rake = Rake(language_code='ru')
def extract_rake(text):
    l=[]
    for keyword in rake.apply(text):
        if keyword[1]>=5:
            for i in keyword[0].split():
                l.append(i)
    return l

In [27]:
extract_rake(data['content'][1])

['новым',
 'витком',
 'цен',
 'озвучил',
 'летом',
 'прошлого',
 'очередные',
 'заявления',
 'представителей',
 'ранее',
 'озвученных',
 'прогнозов',
 'осваивать',
 'новые',
 'регионы',
 'рукаве',
 'сильный',
 'козырь',
 'шельфе',
 'баренцева',
 'моря',
 'периоду',
 'увеличат',
 'производство',
 'согласно',
 'экспертным',
 'оценкам',
 'удар',
 'попадут',
 'прежде',
 'регулируемым',
 'федеральной',
 'службой',
 'условиях',
 'рыночного',
 'ценообразования',
 'концерн',
 'собирается',
 'добывать',
 'холдинг',
 'планирует',
 'вкладываться',
 'общее',
 'производство',
 'газа',
 'треть',
 'добываемого',
 'объема',
 'уточнил',
 'предыдущий',
 'прогноз',
 'минэкономразвития',
 'добыча',
 'газа',
 'суммарная',
 'добыча',
 'газа',
 'сложно',
 'оценить',
 'однозначно',
 'подкорректированные',
 'планы',
 'выглядят',
 'стадии',
 'падающей',
 'добычи',
 'среднемировому',
 'уровню',
 'потребители',
 'обеспечить',
 'независимые',
 'производители',
 'резко',
 'должны',
 'вырасти',
 'прибыльные',
 'эксп

In [29]:
data['rake']=data['content'].apply(extract_rake)

In [30]:
evaluate(data['keywords'], data['rake'])

Precision -  0.01
Recall -  0.14
F1 -  0.02
Jaccard -  0.01


Можно видеть, что ничего особенного из этой затеи не получилось. Попробуем продолжить с графами.

In [78]:
from itertools import combinations

In [79]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [80]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

Wall time: 2min 33s


In [81]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [82]:
keywords_rw.head(10)

0    [яблоко, год, политика, акция, виктор, россия,...
1    [миллиард, добыча, газа, газпром, год, куб, пр...
2    [год, стихотворение, книга, роман, александр, ...
3    [ким, видео, год, журналист, зинаида, адвокат,...
4    [площадь, территория, москва, га, парка, тинао...
5    [гонка, команда, заезд, позиция, пилот, машина...
6    [место, влияние, глава, рф, позиция, институт,...
7    [культура, сфера, закон, концепция, изменение,...
8    [год, поток, газа, катар, европа, газопровод, ...
9    [рф, организация, участник, фсб, область, март...
Name: content_norm, dtype: object

In [83]:
import networkx as nx

In [84]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

## Что-то заработало

Способ 1: поменять аргументы centrality measure

In [93]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))

Wall time: 32.3 s


In [94]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.18
Recall -  0.17
F1 -  0.17
Jaccard -  0.1
